# Import Problem Instance
We start by importing a simple problem instance to demonstrate the tsplib reader.


In [ ]:
from tsplib95 import tsplib95
import itertools
import networkx as nx

instance = tsplib95.load_problem('./tsplib/ulysses16.tsp')

instance.comment

Remember, this repository contains a small selection of TSP instances that you can use to test your algorithms.

| name | nodes | description |
|------|-------|-------------|
| ulysses16.tsp | 16 | Odyssey of Ulysses |
| ulysses7.tsp | 7 | subset of ulysses16 for testing purposes |
| bayg29.tsp | 29 | 29 Cities in Bavaria |
| bier127.tsp | 127 | 127 Biergaerten in Augsburg |
| bier20.tsp | 20 | subset of bier127 |
| brazil58.tsp | 58 | 58 cities in Brazil |
| ali535.tsp | 535 | 535 Airports around the globe |
| d18512.tsp | 18512 | 18512 places in Germany |

The following calls show the dimension = number of nodes of the problem, its node set and the edge weights. The functions `instance.get_nodes()` and `instance.get_edges()` are implemented as iterators, so you can loop over the nodes or edges. To get a list of nodes or edges, you have to explicitly construct one using `list(instance.get_nodes())`. Note that node counting may start at 1 for some instances while others use 0 as starting point. For convenience, we store the index of the first node as `first_node`.


In [ ]:
instance.dimension

instance.get_nodes()
print("List of nodes: ", list(instance.get_nodes()))

first_node = min(instance.get_nodes())
first_node

for i,j in instance.get_edges():
    if i >= j:
        continue
    print(f"edge {{ {i:2},{j:2} }} has weight {instance.wfunc(i,j):3}.")


You have already seen how to draw a graph, here is the relevant code again.

In [ ]:
G = instance.get_graph()
if instance.is_depictable():
    pos = {i: instance.get_display(i) for i in instance.get_nodes()}
else:
    pos = nx.drawing.layout.spring_layout(G)
nx.draw_networkx_nodes(G, pos, node_color='#66a3ff', node_size=200)
nx.draw_networkx_labels(G, pos, font_weight='bold' )
nx.draw_networkx_edges(G, pos,  edge_color='#e6e6e6')

# Implementing a multi commodity flow model in Gurobi
We will implement a multi commodity flow model using binary variables $x_{ij} \in \{0,1\}$ to indicate whether edge $\{i,j\}$ is being used in the tour. For the flow we use variables $y_{i,j,k}$ indicating the flow of commodity $k$ from node $v_0$ into node $k$ along arc $(i,j)$. Flow is only allowed on arcs that are used in the tour and the capacity of each such arc is $1$, we send $2$ units of flow from $v_0$ to $k$. For the formulation, we employ the condition that $i < j$ for any edge $\{i,j\}$ for the $x$-variables. The formulation looks like this:

\begin{align}
\min\;&\sum_{i,j \in E} c_{i,j} \cdot x_{i,j}\\
&\sum_{j \ne i} x_{i,j} = 2 \quad \text{for all nodes $i$}\\
&\sum_{j \ne k} y_{v_0,j,k} = 2 \quad \text{for all nodes $k \ne v_0$}\\
&\sum_{j \ne k} y_{j,v_0,k} = 0 \quad \text{for all nodes $k \ne v_0$}\\
&\sum_{j \ne i} y_{j,i,k} - \sum_{j \ne i} y_{i,j,k} = 0 \quad \text{for all nodes $k \ne v_0$ and all nodes $i \notin \{v_0,k\}$}\\
&y_{i,j,k} \le x_{i,j} \quad \text{for all arcs $(i,j)$ and all $k \ne v_0$}\\
&y_{j,i,k} \le x_{i,j} \quad \text{for all arcs $(i,j)$ and all $k \ne v_0$}\\
&x_{i,j} \in \{0,1\}\\
&y_{i,j,k} \ge 0 \quad \text{for all $k \ne 1$}
\end{align}

## Creating the variables
We start by creating the model and the variables and setting the objective.


In [ ]:
import gurobipy as grb

model = grb.Model(name="Multi Commodity Flow TSP Formulation")
model.reset()

x = model.addVars(filter(lambda e: e[0] < e[1], instance.get_edges()), vtype=grb.GRB.BINARY, name="x")

y = model.addVars(filter(lambda e: e[0] != e[1], instance.get_edges()), range(first_node+1, instance.dimension+1), name="y")

model.setObjective(sum(x[i,j]*instance.wfunc(i,j) for i,j in x.keys()))


## Degree Constraints
We first add the degree constraints for the $x$-variables:
\begin{align}
&\sum_{j \ne i} x_{i,j} = 2 \quad \text{for all nodes $i$}
\end{align}

In [ ]:
for i in instance.get_nodes():
    model.addConstr(x.sum(i,'*') + x.sum('*',i) == 2, name=f"degree[{i}]")

## Flow Constraints
**Task 1:** Add the flow conservations constraints as well as the in- and out-flow constraints for $v_0$ for each commodity k:

## Capacity Constraints
**Task 2:** Add the capacity constraints:
\begin{align}
&y_{i,j,k} \le x_{i,j} \quad \text{for all arcs $(i,j)$ and all $k \ne v_0$}\\
&y_{j,i,k} \le x_{i,j} \quad \text{for all arcs $(i,j)$ and all $k \ne v_0$}
\end{align}

## Starting the Optimization Process
Finally, we set the objective to minimization and call the optimizer.

In [ ]:
model.ModelSense = grb.GRB.MINIMIZE
model.reset()
model.optimize()

## Querying and Visualizing the Solution
Before we visualize our result, let us look at a few key figures of our solution.

In [ ]:
model.ObjVal

solution_edges = [(i,j) for i,j in x.keys() if x[i,j].x > 0.9]
solution_edges

flow_edges = [f"({i},{j},{k}): {y[i,j,k].x}" for i,j,k in y.keys() if y[i,j,k].x > 0]
flow_edges

For debugging purposes, it might be helpful to export the model held by Gurobi into a human-readable format:

In [ ]:
model.write('test.lp')

Finally, let us visualize the solution using NetworkX. In this case, we need to prescribe positions and draw the nodes and two layers of edges separately.

In [ ]:
if instance.is_depictable():
    pos = {i: instance.get_display(i) for i in instance.get_nodes()}
else:
    pos = nx.drawing.layout.spring_layout(G)
nx.draw_networkx_nodes(G, pos, node_color='#66a3ff', node_size=500)
nx.draw_networkx_labels(G, pos, font_weight='bold' )
nx.draw_networkx_edges(G, pos,  edge_color='#e6e6e6')
nx.draw_networkx_edges(G, pos,  edgelist=solution_edges, edge_color='#ffa31a', width=4)

**Task 3:** Test the model for different instances. How does it compare to the MTZ formulation?